In [1]:
# basic imports for web-scraping
# note to self: use Python 3.9.0

from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import pytube
from urllib3 import Timeout
import json, time

In [35]:
def youtube_scrape(search_query, num_results): # retrieving results from youtube
  driver = webdriver.Chrome(r'\\chromedriver')
  results_links = []

  link = "https://www.youtube.com/results?search_query=" + search_query
  driver.get(link)

  start_time = time.time()
  delta_time = time.time() - start_time

  while delta_time < 5: #loop loads elements for 5 seconds or until last result is reached (edge case)
    end_result = driver.find_element(By.CSS_SELECTOR, '#message').is_displayed()
    driver.execute_script("var scrollingElement = (document.scrollingElement || document.body);scrollingElement.scrollTop = scrollingElement.scrollHeight;")
    # print(end_result)
    delta_time = time.time() - start_time
    print("DELTA TIME: " + str(delta_time))
    if (end_result): 
      break
  
  print("Extracting results...")
  i = 1
  for result in driver.find_elements(By.CSS_SELECTOR, '.text-wrapper.style-scope.ytd-video-renderer'):
    link = result.find_element(By.CSS_SELECTOR, '.title-and-badge.style-scope.ytd-video-renderer a').get_attribute('href')

    results_links.append(link)
    i += 1
    if (i > num_results):
      break

  if (len(results_links) > 0):
    print(results_links[0])
  print("LENGTH OF RESULTS: " + str(len(results_links)))

  return results_links



In [37]:
# searching youtube for the listed queries and taking weighted amounts of each (takes approx. 3 mins)

search_queries = ["running form", "bad running form example", "running form mistakes posetv", "running overstriding example", "running overpulling example", 
                    "heel-strike running example", "poor running form", "marathon highlights", "cross country running", "5000m race", "10000m race", "kenya track workout", 
                    "paul chelimo", "timothy cheruiyot", "long distance running luis orta", "kitata running form", "elite long run", "sweat elite training sessions"]
search_weights = [10, 20, 10, 10, 10, 10, 20, 10, 40, 20, 20, 20, 20, 20, 20, 5, 10, 20]

# file = open(".\\Web_Scrape_Precut\\web-scraped-links.txt")

res = [] # list holding all results
for i in range(len(search_queries)):
    # print(i)
    temp_res = youtube_scrape(search_queries[i], search_weights[i])
    for j in range(len(temp_res)):
        res.append(temp_res[j])

print("\n\nFINAL RESULTS\n\n")
for j in range(len(res)):
    print(res[j])
    print("\n")



C:\Users\adars\AppData\Local\Temp\ipykernel_23036\1641197535.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r'\\chromedriver')


DELTA TIME: 1.5104668140411377
DELTA TIME: 2.8055999279022217
DELTA TIME: 3.9418184757232666
DELTA TIME: 4.690852880477905
DELTA TIME: 5.668489933013916
Extracting results...
https://www.youtube.com/watch?v=brFHyOtTwH4
LENGTH OF RESULTS: 10
DELTA TIME: 1.0430257320404053
DELTA TIME: 2.247493028640747
DELTA TIME: 3.055236577987671
DELTA TIME: 3.627664566040039
DELTA TIME: 4.844532489776611
DELTA TIME: 5.48011589050293
Extracting results...
https://www.youtube.com/watch?v=qM1aYKM6y2M
LENGTH OF RESULTS: 20
DELTA TIME: 0.9518296718597412
DELTA TIME: 1.6940343379974365
DELTA TIME: 2.6872236728668213
DELTA TIME: 3.2122154235839844
DELTA TIME: 3.989372730255127
DELTA TIME: 4.638684511184692
DELTA TIME: 6.343928098678589
Extracting results...
https://www.youtube.com/watch?v=JcX4CDhqo04
LENGTH OF RESULTS: 10
DELTA TIME: 0.7079272270202637
DELTA TIME: 1.828026294708252
DELTA TIME: 2.616797685623169
DELTA TIME: 3.2808642387390137
DELTA TIME: 3.8337762355804443
DELTA TIME: 4.912908315658569
DELTA 

KeyboardInterrupt: 

In [ ]:
# downloading the videos from youtube


def videoName(i):
    ret = str(i)
    while (len(ret) < 4):
        ret = "0" + ret
    return ret


counter = 0

for video in res:
    try:
        with Timeout(600):
            url = video['link']
            yt = pytube.YouTube(url)
            video = yt.streams.first()

            video.download(output_path=".\\Web_Scrape_Precut\\", filename=videoName(counter))
    except:
        print("Failed to download video number " + str(videoName(counter)))
    counter += 1
